# 영화 추천 시스템
## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2. 데이터 분석

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [10]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화 추가

In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('mission', regex=False)]

,movie_id,title,genre
642,648,Mission: Impossible (1996),Action|Adventure|Mystery
2676,2745,"Mission, The (1986)",Drama
3285,3354,Mission to Mars (2000),Sci-Fi
3554,3623,Mission: Impossible 2 (2000),Action|Thriller
3817,3887,Went to Coney Island on a Mission From God... ...,Drama


In [14]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [1721, 3578, 924 , 1584, 648]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'eunchae'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['eunchae']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,eunchae,1721,5,Titanic (1997)
1,eunchae,3578,5,Gladiator (2000)
2,eunchae,924,5,2001: A Space Odyssey (1968)
3,eunchae,1584,5,Contact (1997)
4,eunchae,648,5,Mission: Impossible (1996)


In [15]:
if not ratings.isin({'user_id':['eunchae']})['user_id'].any():  # user_id에 'eunchae'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,eunchae,1721,5,Titanic (1997)
836479,eunchae,3578,5,Gladiator (2000)
836480,eunchae,924,5,2001: A Space Odyssey (1968)
836481,eunchae,1584,5,Contact (1997)
836482,eunchae,648,5,Mission: Impossible (1996)


## 4. indexing 후 CSR matrix 생성

In [16]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,1721,5,27
836479,6039,3578,5,67
836480,6039,924,5,602
836481,6039,1584,5,479


In [18]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련시키기

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [24]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
eunchae, titanic = user_to_idx['eunchae'], movie_to_idx['Titanic (1997)']
eunchae_vector, titanic_vector = als_model.user_factors[eunchae], als_model.item_factors[titanic]

In [25]:
eunchae_vector   # 나의 벡터

array([ 0.9010512 ,  0.06293321, -0.05400103, -1.0405308 ,  0.11611883,
        0.53306115, -0.13413236,  0.3516006 ,  0.33071047,  0.71742785,
        0.33420658,  0.5441171 ,  0.6227914 , -0.93562907,  0.0265533 ,
       -0.13166341,  0.36607385,  0.02724123, -0.19291481, -0.54566455,
        0.27910978, -0.18051599,  0.29871905,  1.0946039 , -0.0025848 ,
        0.00340071, -0.83310634, -1.1984048 ,  0.1918616 ,  0.13393945,
        0.8239284 ,  0.44783565,  0.26030886,  0.7648104 ,  0.07367312,
        0.62130016,  0.5861572 , -0.01599549, -0.4333647 ,  0.67340475,
       -0.15155056,  0.09298727,  0.05054825,  0.1993632 ,  1.1826173 ,
        0.19567344,  0.82371044, -0.00465258, -0.5655628 , -0.14521   ,
       -0.49934503, -0.6074964 , -0.17337693, -0.55346644, -0.00689804,
       -0.2983391 , -0.41227806, -0.68935275,  0.9268794 , -0.06738505,
        0.11894057,  0.38009372,  0.67533755, -0.43511894, -1.2345277 ,
        0.02049535,  0.79388565,  0.3822172 ,  1.2359475 , -0.02

In [26]:
titanic_vector   # 영화 타이타닉의 벡터

array([ 0.00527524,  0.01145481,  0.02088631,  0.00215262,  0.01343764,
        0.03573684,  0.01907145, -0.03776411,  0.00906103,  0.04237764,
        0.013081  ,  0.04180159,  0.01427194, -0.00536222,  0.01796505,
       -0.01731455,  0.01269065,  0.00536081,  0.00961554, -0.00453164,
       -0.00697739, -0.02137148,  0.02262677,  0.04225931, -0.00535811,
        0.02554387,  0.00772405, -0.02316961,  0.03895612,  0.01901181,
        0.02988115, -0.03265673, -0.00695818,  0.01558275, -0.00268338,
        0.00572487,  0.00592594, -0.04644748, -0.02385505,  0.01372075,
       -0.00493003,  0.00933489,  0.03724795,  0.00234692,  0.02750989,
        0.00536514,  0.03345224,  0.03034244, -0.02667755, -0.01192702,
        0.00786974,  0.01700561,  0.02147213, -0.03849904,  0.04165869,
       -0.02797903, -0.02442903, -0.00425606,  0.02104322,  0.01135108,
        0.0254058 ,  0.00299712,  0.00691507,  0.01876099, -0.03775723,
        0.00250338,  0.02940518,  0.02378713,  0.04084168, -0.02

In [27]:
# 나와 타이타닉의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(eunchae_vector, titanic_vector)

0.648702

In [28]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['Matrix, The (1999)']
matrix_vector = als_model.item_factors[matrix]
np.dot(eunchae_vector, matrix_vector)

0.17278683

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [29]:
favorite_movie = 'Mission: Impossible (1996)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(58, 0.9999999),
 (138, 0.54077554),
 (82, 0.4739479),
 (118, 0.46397078),
 (766, 0.46285725),
 (587, 0.4521367),
 (179, 0.44558442),
 (129, 0.44282746),
 (156, 0.4395322),
 (714, 0.4141412),
 (421, 0.39647272),
 (67, 0.38466066),
 (71, 0.3834644),
 (816, 0.38228446),
 (140, 0.38197434)]

In [30]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Lost World: Jurassic Park, The (1997)',
 'Con Air (1997)',
 'GoldenEye (1995)',
 'Batman Returns (1992)',
 'Rock, The (1996)',
 'Conspiracy Theory (1997)',
 'Mask of Zorro, The (1998)',
 'Waterworld (1995)',
 'Midnight in the Garden of Good and Evil (1997)',
 'Gladiator (2000)',
 'Snake Eyes (1998)',
 'Air Force One (1997)',
 'Clear and Present Danger (1994)']

In [31]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [32]:
get_similar_movie('Gladiator (2000)')

['Gladiator (2000)',
 'Patriot, The (2000)',
 'Mission: Impossible 2 (2000)',
 'Perfect Storm, The (2000)',
 'Frequency (2000)',
 'U-571 (2000)',
 'X-Men (2000)',
 'Gone in 60 Seconds (2000)',
 'Shanghai Noon (2000)',
 'Mission: Impossible (1996)']

In [33]:
get_similar_movie('2001: A Space Odyssey (1968)')

['2001: A Space Odyssey (1968)',
 'Close Encounters of the Third Kind (1977)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Clockwork Orange, A (1971)',
 'Alien (1979)',
 'Blade Runner (1982)',
 '2010 (1984)',
 'Brazil (1985)',
 '8MM (1999)',
 'Follow the Bitch (1998)']

## 8. 내가 가장 좋아할 만한 영화 추천

In [34]:
user = user_to_idx['eunchae']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(463, 0.3661592),
 (91, 0.32686985),
 (346, 0.31068605),
 (482, 0.27236742),
 (3, 0.26868504),
 (746, 0.267873),
 (680, 0.23143463),
 (384, 0.22003835),
 (317, 0.2196024),
 (550, 0.21872547),
 (1537, 0.21720678),
 (904, 0.21399364),
 (1097, 0.21167754),
 (372, 0.19680373),
 (357, 0.19579004),
 (193, 0.19326779),
 (1352, 0.1920004),
 (670, 0.19184102),
 (138, 0.19172597),
 (39, 0.19037847)]

In [35]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Frequency (2000)',
 'Close Encounters of the Third Kind (1977)',
 'Patriot, The (2000)',
 'Mission: Impossible 2 (2000)',
 'Erin Brockovich (2000)',
 'Perfect Storm, The (2000)',
 'Blade Runner (1982)',
 'Jerry Maguire (1996)',
 'Twelve Monkeys (1995)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'U-571 (2000)',
 'Clockwork Orange, A (1971)',
 'Brazil (1985)',
 'X-Men (2000)',
 'Sense and Sensibility (1995)',
 'Alien (1979)',
 'Sliding Doors (1998)',
 'Galaxy Quest (1999)',
 'True Lies (1994)',
 'Apollo 13 (1995)']

In [49]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Blade Runner (1982)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('2001: A Space Odyssey (1968)', 0.1515442212436981),
 ('Contact (1997)', 0.031651570201456376),
 ('Gladiator (2000)', 0.027373559084833982),
 ('Mission: Impossible (1996)', 0.020095297976737304),
 ('Titanic (1997)', -0.0010807070232094471)]

***
# 프로젝트 정리
- **데이터 전처리**: 영화 평점과 제목이 다른 파일로 나뉘어 있어서 어떻게 처리해야 할지 고민했다. 이런저런 방법을 시도해봤는데, 어차피 나중에 영화 제목을 계속 사용하기 때문에 그때마다 참조하긴 번거로워서 두 데이터프레임을 join하는 방법을 택했다.
- **CSR matrix 생성**: matrix를 만들 때 계속 오류가 발생했다. user_id와 movie_id 값이 숫자여서 이미 인덱싱이 되어 있다고 생각했고, 좋아하는 영화 5개를 추가하는 부분에서 user_id와 movie_id를 숫자로 추가했는데도 에러가 났다. 알고보니 데이터가 숫자처럼 보이지만 사실 제대로 인덱싱이 되어 있지 않아서 오류가 생긴 것이었다. 실제로 인덱싱 과정을 거치니 matrix가 제대로 생성되었다.
- **결과 분석**: favorite 리스트에 넣었던 영화 '타이타닉'과의 벡터 내적은 0.65로 비교적 높았고, 선호 리스트에 넣지 않은 영화 '매트릭스'와의 벡터 내적은 0.17로 비교적 낮았다. 내가 좋아하는 영화와 비슷한 영화 추천도 액션이면 액션, SF면 SF 등 유사한 장르의 영화가 추천되었다. 그리고 내가 좋아할 만한 영화 추천 목록에도 실제로 보고 싶은 영화들이 포함되어 있어 추천 모델의 성능이 꽤 괜찮은 것 같았다.